In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from gridding_python_improved.gridding import gcf_kaiser, gcf_kaiseri

In [3]:
def grid_1d_from_2d(x, vis, dx, W, beta, y, x2, vis2, y2):
    """
    Grid the data in w, Qvix, Uvis in 1D (x) and duplicate orthogonal axes
    """
    N = len(x)

    Dx = W*dx
        
    for indx in range(N):

        visval = vis[indx]

        xval = x[indx]
        yval = y[indx]
        
        xref = np.ceil((xval - 0.5*W*dx)/dx)*dx

        for xndx in range(W):
            
            xg = xref + xndx*dx
            
            kndx = indx*W + xndx
            
            gcf_val = gcf_kaiser(xg-xval, Dx, beta)
            
            vis2[kndx] = visval*gcf_val

            x2[kndx] = xg
            y2[kndx] = yval
            
    
    print(x2.shape, y2.shape, vis2.shape)
            


In [4]:
def grid_1d_from_2di(x, vis, dx, W, beta, y, x2, vis2, y2):
    N = len(x)
    Dx = W * dx
    visval = vis
    xval = x
    yval = y
    xref = np.ceil((xval - 0.5*W*dx)/dx)*dx
    xndx = np.arange(W)
    xg = xref + xndx * dx
    gcf_val = gcf_kaiseri(xg - xval, Dx, beta)
    vis2 = np.outer(visval, gcf_val).ravel()
    
    x2 = np.tile(xg, N)
    y2 = np.repeat(yval, W)
    
    return x2, vis2, y2

        

In [5]:
y2 = np.zeros([36])
xndx = np.arange(36).reshape(6, 6)
yval = np.arange(6)
y2[xndx] = yval

In [6]:
y2

array([0., 1., 2., 3., 4., 5., 0., 1., 2., 3., 4., 5., 0., 1., 2., 3., 4.,
       5., 0., 1., 2., 3., 4., 5., 0., 1., 2., 3., 4., 5., 0., 1., 2., 3.,
       4., 5.])

In [7]:
nvis = 1
u = np.random.randn(nvis)
v = np.random.randn(nvis)
vis = np.random.randn(nvis) * np.random.randn(nvis) * 1j
du = 2/3
Nu = 3450
umin=-1150
dv = 2/3
Nv = 3450
umin=-1150
alpha = 1.5
W = 6
hflag_u = True
hflag_v = True
beta = np.array([12.24])


tu1 = np.zeros(W)
tvis1 = np.zeros(W, dtype=complex)
tv1 = np.zeros(W)


tu1i = np.zeros(W)
tvis1i = np.zeros(W, dtype=complex)
tv1i = np.zeros(W)

su = np.zeros(1)
sv = np.zeros(1)
svis = np.zeros(1, dtype=complex)

W2 = W**2
nvis = u.shape[0]

tu2 = np.zeros(W2)
tvis2 = np.zeros(W2, dtype=complex)
tv2 = np.zeros(W2)

tu2i = np.zeros(W2)
tvis2i = np.zeros(W2, dtype=complex)
tv2i = np.zeros(W2)


for i in range(nvis):
    su[0] = u[i]
    sv[0] = v[i]
    svis[0] = vis[i]
    grid_1d_from_2d(su, svis, du, W, beta, sv, tu1, tvis1, tv1)
    
    grid_1d_from_2d(tv1, tvis1, dv, W, beta, tu1, tv2, tvis2, tu2) # output arrays

    tu1i, tvis1i, tv1i = grid_1d_from_2di(su, svis, du, W, beta, sv, tu1i, tvis1i, tv1i)
    
    tv2i, tvis2i, tu2i = grid_1d_from_2di(tv1i, tvis1i, dv, W, beta, tu1i, tv2i, tvis2i, tu2i) # output arrays
        
    if not (tv1 == tv1i).all():
        print(tv1, tv1i)
        raise
        
    if not (tv2 == tv2i).all():
        print(tv2, tv2i)
        raise
        
    if not (tvis1 == tvis1i).all():
        print(tvis1, tvis1i)
        raise
        
    if not (tvis2 == tvis2i).all():
        print(tvis1, tvis1i)
        raise
        
    if not (tu1 == tu1i).all():
        print(tu1, tu1i)
        
        raise
    
    if not (tu2 == tu2i).all():
        print(tu2, tu2i)
        raise



#     print((tv1 == tv1i).all())
#     print(tv2.sum(), tv2i.sum())
#     print((tv2 == tv2i).all(), tv2.sum() == tv2i.sum())
#     print(tvis1.sum())
#     print(tu1.sum())
#     print(tv1.sum())
    
#     print(tvis2.sum())
#     print(tu2.sum())
#     print(tv2.sum())
    
#     print('\n')
       
#     print(tvis1i.sum())
#     print(tu1i.sum())
#     print(tv1i.sum())
    
#     print(tvis2i.sum())
#     print(tu2i.sum())
#     print(tv2i.sum())

(6,) (6,) (6,)
(36,) (36,) (36,)


In [8]:
tv2

array([0.66666667, 1.33333333, 2.        , 2.66666667, 3.33333333,
       4.        , 0.66666667, 1.33333333, 2.        , 2.66666667,
       3.33333333, 4.        , 0.66666667, 1.33333333, 2.        ,
       2.66666667, 3.33333333, 4.        , 0.66666667, 1.33333333,
       2.        , 2.66666667, 3.33333333, 4.        , 0.66666667,
       1.33333333, 2.        , 2.66666667, 3.33333333, 4.        ,
       0.66666667, 1.33333333, 2.        , 2.66666667, 3.33333333,
       4.        ])

In [9]:
tv2i

array([0.66666667, 1.33333333, 2.        , 2.66666667, 3.33333333,
       4.        , 0.66666667, 1.33333333, 2.        , 2.66666667,
       3.33333333, 4.        , 0.66666667, 1.33333333, 2.        ,
       2.66666667, 3.33333333, 4.        , 0.66666667, 1.33333333,
       2.        , 2.66666667, 3.33333333, 4.        , 0.66666667,
       1.33333333, 2.        , 2.66666667, 3.33333333, 4.        ,
       0.66666667, 1.33333333, 2.        , 2.66666667, 3.33333333,
       4.        ])

In [10]:
x = np.arange(5)

In [11]:
np.tile(x, 6)

array([0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1,
       2, 3, 4, 0, 1, 2, 3, 4])

In [66]:
nvis = 2
u = np.random.randn(1).repeat(nvis)
v = np.random.randn(1).repeat(nvis)
vis = np.random.randn(1).repeat(nvis) * np.random.randn(1).repeat(nvis) * 1j
du = 2/3
Nu = 8
umin=-3
dv = 2/3
Nv = 8
vmin=-3
alpha = 1.5
W = 6
hflag_u = True
hflag_v = True
beta = np.array([12.24])

ug = np.zeros(nvis*W2)
vg = np.zeros(nvis*W2)
visg = np.zeros(nvis*W2, dtype=complex)


tu1 = np.zeros(W)
tvis1 = np.zeros(W, dtype=complex)
tv1 = np.zeros(W)


tu1i = np.zeros(W)
tvis1i = np.zeros(W, dtype=complex)
tv1i = np.zeros(W)

su = np.zeros(1)
sv = np.zeros(1)
svis = np.zeros(1, dtype=complex)

W2 = W**2
nvis = u.shape[0]

tu2 = np.zeros(W2)
tvis2 = np.zeros(W2, dtype=complex)
tv2 = np.zeros(W2)

tu2i = np.zeros(W2)
tvis2i = np.zeros(W2, dtype=complex)
tv2i = np.zeros(W2)


for i in range(nvis):
    su[0] = u[i]
    sv[0] = v[i]
    svis[0] = vis[i]
#     grid_1d_from_2d(su, svis, du, W, beta, sv, tu1, tvis1, tv1)
    
#     grid_1d_from_2d(tv1, tvis1, dv, W, beta, tu1, tv2, tvis2, tu2) # output arrays
   
#     ug[i*W2:(i+1)*W2] = tu2
#     vg[i*W2:(i+1)*W2] = tv2
#     visg[i*W2:(i+1)*W2] = tvis2

    tu1i, tvis1i, tv1i = grid_1d_from_2di(su, svis, du, W, beta, sv, tu1i, tvis1i, tv1i)
    
    tv2i, tvis2i, tu2i = grid_1d_from_2di(tv1i, tvis1i, dv, W, beta, tu1i, tv2i, tvis2i, tu2i) # output arrays
        
    ug[i*W2:(i+1)*W2] = tu2i
    vg[i*W2:(i+1)*W2] = tv2i
    visg[i*W2:(i+1)*W2] = tvis2i



In [67]:
ugii = np.zeros(nvis*W2)
vgii = np.zeros(nvis*W2)
visgii = np.zeros(nvis*W2, dtype=complex)


tu1ii = np.zeros([nvis, W])
tvis1ii = np.zeros([nvis, W], dtype=complex)
tv1ii = np.zeros([nvis, W])

W2 = W**2
nvis = u.shape[0]

tu2ii = np.zeros([nvis, W2])
tvis2ii = np.zeros([nvis, W2], dtype=complex)
tv2ii = np.zeros([nvis, W2])




In [68]:
def grid_1d_from_2dii(x, vis, dx, W, beta, y, x2, vis2, y2):
    nvis, N = x.shape
    Dx = W * dx
    visval = vis
    xval = x
    yval = y
    xref = np.ceil((xval - 0.5*W*dx)/dx)*dx
    xndx = np.arange(W)

    xg = xref + xndx * dx
    gcf_val = gcf_kaiseri(xg - xval, Dx, beta)
    
    vis2 = np.matmul(visval[:, :, None], gcf_val[:, None, :])
    vis2 = vis2.reshape(nvis, -1)
    x2 = np.tile(xg, N)
    y2 = np.repeat(yval, W, axis=-1)
    
    return x2, vis2, y2

tu1ii, tvis1ii, tv1ii = grid_1d_from_2dii(u[:, None], vis[:, None], du, W, beta, v[:, None], tu1ii, tvis1ii, tv1ii)


In [69]:
assert np.all(tu1ii[-1] == tu1i)
assert np.all(tvis1ii[-1] == tvis1i)
assert np.all(tv1ii[-1] == tv1i)

In [70]:
u[:, None].shape, vis[:, None].shape, v[:, None].shape, tu1ii.shape, tvis1ii.shape, tv1ii.shape

((2, 1), (2, 1), (2, 1), (2, 6), (2, 6), (2, 6))

In [71]:
tv1ii.shape, tvis1ii.shape, tu1ii.shape, tv2ii.shape, tvis2ii.shape, tu2ii.shape

((2, 6), (2, 6), (2, 6), (2, 36), (2, 36), (2, 36))

In [72]:
tv2ii, tvis2ii, tu2ii = grid_1d_from_2dii(tv1ii, tvis1ii, dv, W, beta, tu1ii, tv2ii, tvis2ii, tu2ii) # output arrays


In [73]:
assert np.all(tu2ii[-1] == tu2i)
assert np.all(tvis2ii[-1] == tvis2i)
assert np.all(tv2ii[-1] == tv2i)

In [74]:
ugii = tu2ii.ravel()
vgii = tv2ii.ravel()
visgii = tvis2ii.ravel()

In [75]:
visgii == visg

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True])

In [76]:
visg

array([0.-4.32616137e-06j, 0.-1.34823042e-04j, 0.-5.07402574e-04j,
       0.-4.75564372e-04j, 0.-1.08201624e-04j, 0.-2.47714661e-06j,
       0.-9.87490284e-04j, 0.-3.07747291e-02j, 0.-1.15819793e-01j,
       0.-1.08552400e-01j, 0.-2.46981201e-02j, 0.-5.65433880e-04j,
       0.-7.49414578e-03j, 0.-2.33551975e-01j, 0.-8.78966025e-01j,
       0.-8.23813176e-01j, 0.-1.87436084e-01j, 0.-4.29112468e-03j,
       0.-1.21547512e-02j, 0.-3.78797829e-01j, 0.-1.42559455e+00j,
       0.-1.33614217e+00j, 0.-3.04002490e-01j, 0.-6.95977293e-03j,
       0.-5.14000645e-03j, 0.-1.60186190e-01j, 0.-6.02856038e-01j,
       0.-5.65028378e-01j, 0.-1.28556704e-01j, 0.-2.94315178e-03j,
       0.-3.84378173e-04j, 0.-1.19789879e-02j, 0.-4.50825704e-02j,
       0.-4.22537555e-02j, 0.-9.61368270e-03j, 0.-2.20093752e-04j,
       0.-4.32616137e-06j, 0.-1.34823042e-04j, 0.-5.07402574e-04j,
       0.-4.75564372e-04j, 0.-1.08201624e-04j, 0.-2.47714661e-06j,
       0.-9.87490284e-04j, 0.-3.07747291e-02j, 0.-1.15819793e-

In [93]:
from numpy import array

In [215]:
ug, vg, visg = (array([-2.        , -2.        , -2.        , -2.        , -2.        ,
       -2.        , -1.33333333, -1.33333333, -1.33333333, -1.33333333,
       -1.33333333, -1.33333333, -0.66666667, -0.66666667, -0.66666667,
       -0.66666667, -0.66666667, -0.66666667,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.66666667,
        0.66666667,  0.66666667,  0.66666667,  0.66666667,  0.66666667,
        1.33333333,  1.33333333,  1.33333333,  1.33333333,  1.33333333,
        1.33333333, -2.66666667, -2.66666667, -2.66666667, -2.66666667,
       -2.66666667, -2.66666667, -2.        , -2.        , -2.        ,
       -2.        , -2.        , -2.        , -1.33333333, -1.33333333,
       -1.33333333, -1.33333333, -1.33333333, -1.33333333, -0.66666667,
       -0.66666667, -0.66666667, -0.66666667, -0.66666667, -0.66666667,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.66666667,  0.66666667,  0.66666667,  0.66666667,
        0.66666667,  0.66666667, -2.        , -2.        , -2.        ,
       -2.        , -2.        , -2.        , -1.33333333, -1.33333333,
       -1.33333333, -1.33333333, -1.33333333, -1.33333333, -0.66666667,
       -0.66666667, -0.66666667, -0.66666667, -0.66666667, -0.66666667,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.66666667,  0.66666667,  0.66666667,  0.66666667,
        0.66666667,  0.66666667,  1.33333333,  1.33333333,  1.33333333,
        1.33333333,  1.33333333,  1.33333333,  0.66666667,  0.66666667,
        0.66666667,  0.66666667,  0.66666667,  0.66666667,  1.33333333,
        1.33333333,  1.33333333,  1.33333333,  1.33333333,  1.33333333,
        2.        ,  2.        ,  2.        ,  2.        ,  2.        ,
        2.        ,  2.66666667,  2.66666667,  2.66666667,  2.66666667,
        2.66666667,  2.66666667,  3.33333333,  3.33333333,  3.33333333,
        3.33333333,  3.33333333,  3.33333333,  4.        ,  4.        ,
        4.        ,  4.        ,  4.        ,  4.        ,  1.33333333,
        1.33333333,  1.33333333,  1.33333333,  1.33333333,  1.33333333,
        2.        ,  2.        ,  2.        ,  2.        ,  2.        ,
        2.        ,  2.66666667,  2.66666667,  2.66666667,  2.66666667,
        2.66666667,  2.66666667,  3.33333333,  3.33333333,  3.33333333,
        3.33333333,  3.33333333,  3.33333333,  4.        ,  4.        ,
        4.        ,  4.        ,  4.        ,  4.        ,  4.66666667,
        4.66666667,  4.66666667,  4.66666667,  4.66666667,  4.66666667,
       -2.        , -2.        , -2.        , -2.        , -2.        ,
       -2.        , -1.33333333, -1.33333333, -1.33333333, -1.33333333,
       -1.33333333, -1.33333333, -0.66666667, -0.66666667, -0.66666667,
       -0.66666667, -0.66666667, -0.66666667,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.66666667,
        0.66666667,  0.66666667,  0.66666667,  0.66666667,  0.66666667,
        1.33333333,  1.33333333,  1.33333333,  1.33333333,  1.33333333,
        1.33333333,  6.66666667,  6.66666667,  6.66666667,  6.66666667,
        6.66666667,  6.66666667,  7.33333333,  7.33333333,  7.33333333,
        7.33333333,  7.33333333,  7.33333333,  8.        ,  8.        ,
        8.        ,  8.        ,  8.        ,  8.        ,  8.66666667,
        8.66666667,  8.66666667,  8.66666667,  8.66666667,  8.66666667,
        9.33333333,  9.33333333,  9.33333333,  9.33333333,  9.33333333,
        9.33333333, 10.        , 10.        , 10.        , 10.        ,
       10.        , 10.        ,  8.        ,  8.        ,  8.        ,
        8.        ,  8.        ,  8.        ,  8.66666667,  8.66666667,
        8.66666667,  8.66666667,  8.66666667,  8.66666667,  9.33333333,
        9.33333333,  9.33333333,  9.33333333,  9.33333333,  9.33333333,
       10.        , 10.        , 10.        , 10.        , 10.        ,
       10.        , 10.66666667, 10.66666667, 10.66666667, 10.66666667,
       10.66666667, 10.66666667, 11.33333333, 11.33333333, 11.33333333,
       11.33333333, 11.33333333, 11.33333333,  4.66666667,  4.66666667,
        4.66666667,  4.66666667,  4.66666667,  4.66666667,  5.33333333,
        5.33333333,  5.33333333,  5.33333333,  5.33333333,  5.33333333,
        6.        ,  6.        ,  6.        ,  6.        ,  6.        ,
        6.        ,  6.66666667,  6.66666667,  6.66666667,  6.66666667,
        6.66666667,  6.66666667,  7.33333333,  7.33333333,  7.33333333,
        7.33333333,  7.33333333,  7.33333333,  8.        ,  8.        ,
        8.        ,  8.        ,  8.        ,  8.        , -2.        ,
       -2.        , -2.        , -2.        , -2.        , -2.        ,
       -1.33333333, -1.33333333, -1.33333333, -1.33333333, -1.33333333,
       -1.33333333, -0.66666667, -0.66666667, -0.66666667, -0.66666667,
       -0.66666667, -0.66666667,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.66666667,  0.66666667,
        0.66666667,  0.66666667,  0.66666667,  0.66666667,  1.33333333,
        1.33333333,  1.33333333,  1.33333333,  1.33333333,  1.33333333]), array([-2.        , -1.33333333, -0.66666667,  0.        ,  0.66666667,
        1.33333333, -2.        , -1.33333333, -0.66666667,  0.        ,
        0.66666667,  1.33333333, -2.        , -1.33333333, -0.66666667,
        0.        ,  0.66666667,  1.33333333, -2.        , -1.33333333,
       -0.66666667,  0.        ,  0.66666667,  1.33333333, -2.        ,
       -1.33333333, -0.66666667,  0.        ,  0.66666667,  1.33333333,
       -2.        , -1.33333333, -0.66666667,  0.        ,  0.66666667,
        1.33333333, -0.66666667,  0.        ,  0.66666667,  1.33333333,
        2.        ,  2.66666667, -0.66666667,  0.        ,  0.66666667,
        1.33333333,  2.        ,  2.66666667, -0.66666667,  0.        ,
        0.66666667,  1.33333333,  2.        ,  2.66666667, -0.66666667,
        0.        ,  0.66666667,  1.33333333,  2.        ,  2.66666667,
       -0.66666667,  0.        ,  0.66666667,  1.33333333,  2.        ,
        2.66666667, -0.66666667,  0.        ,  0.66666667,  1.33333333,
        2.        ,  2.66666667, -2.        , -1.33333333, -0.66666667,
        0.        ,  0.66666667,  1.33333333, -2.        , -1.33333333,
       -0.66666667,  0.        ,  0.66666667,  1.33333333, -2.        ,
       -1.33333333, -0.66666667,  0.        ,  0.66666667,  1.33333333,
       -2.        , -1.33333333, -0.66666667,  0.        ,  0.66666667,
        1.33333333, -2.        , -1.33333333, -0.66666667,  0.        ,
        0.66666667,  1.33333333, -2.        , -1.33333333, -0.66666667,
        0.        ,  0.66666667,  1.33333333,  4.66666667,  5.33333333,
        6.        ,  6.66666667,  7.33333333,  8.        ,  4.66666667,
        5.33333333,  6.        ,  6.66666667,  7.33333333,  8.        ,
        4.66666667,  5.33333333,  6.        ,  6.66666667,  7.33333333,
        8.        ,  4.66666667,  5.33333333,  6.        ,  6.66666667,
        7.33333333,  8.        ,  4.66666667,  5.33333333,  6.        ,
        6.66666667,  7.33333333,  8.        ,  4.66666667,  5.33333333,
        6.        ,  6.66666667,  7.33333333,  8.        ,  4.        ,
        4.66666667,  5.33333333,  6.        ,  6.66666667,  7.33333333,
        4.        ,  4.66666667,  5.33333333,  6.        ,  6.66666667,
        7.33333333,  4.        ,  4.66666667,  5.33333333,  6.        ,
        6.66666667,  7.33333333,  4.        ,  4.66666667,  5.33333333,
        6.        ,  6.66666667,  7.33333333,  4.        ,  4.66666667,
        5.33333333,  6.        ,  6.66666667,  7.33333333,  4.        ,
        4.66666667,  5.33333333,  6.        ,  6.66666667,  7.33333333,
       -2.        , -1.33333333, -0.66666667,  0.        ,  0.66666667,
        1.33333333, -2.        , -1.33333333, -0.66666667,  0.        ,
        0.66666667,  1.33333333, -2.        , -1.33333333, -0.66666667,
        0.        ,  0.66666667,  1.33333333, -2.        , -1.33333333,
       -0.66666667,  0.        ,  0.66666667,  1.33333333, -2.        ,
       -1.33333333, -0.66666667,  0.        ,  0.66666667,  1.33333333,
       -2.        , -1.33333333, -0.66666667,  0.        ,  0.66666667,
        1.33333333,  6.66666667,  7.33333333,  8.        ,  8.66666667,
        9.33333333, 10.        ,  6.66666667,  7.33333333,  8.        ,
        8.66666667,  9.33333333, 10.        ,  6.66666667,  7.33333333,
        8.        ,  8.66666667,  9.33333333, 10.        ,  6.66666667,
        7.33333333,  8.        ,  8.66666667,  9.33333333, 10.        ,
        6.66666667,  7.33333333,  8.        ,  8.66666667,  9.33333333,
       10.        ,  6.66666667,  7.33333333,  8.        ,  8.66666667,
        9.33333333, 10.        ,  6.        ,  6.66666667,  7.33333333,
        8.        ,  8.66666667,  9.33333333,  6.        ,  6.66666667,
        7.33333333,  8.        ,  8.66666667,  9.33333333,  6.        ,
        6.66666667,  7.33333333,  8.        ,  8.66666667,  9.33333333,
        6.        ,  6.66666667,  7.33333333,  8.        ,  8.66666667,
        9.33333333,  6.        ,  6.66666667,  7.33333333,  8.        ,
        8.66666667,  9.33333333,  6.        ,  6.66666667,  7.33333333,
        8.        ,  8.66666667,  9.33333333,  0.        ,  0.66666667,
        1.33333333,  2.        ,  2.66666667,  3.33333333,  0.        ,
        0.66666667,  1.33333333,  2.        ,  2.66666667,  3.33333333,
        0.        ,  0.66666667,  1.33333333,  2.        ,  2.66666667,
        3.33333333,  0.        ,  0.66666667,  1.33333333,  2.        ,
        2.66666667,  3.33333333,  0.        ,  0.66666667,  1.33333333,
        2.        ,  2.66666667,  3.33333333,  0.        ,  0.66666667,
        1.33333333,  2.        ,  2.66666667,  3.33333333, -2.        ,
       -1.33333333, -0.66666667,  0.        ,  0.66666667,  1.33333333,
       -2.        , -1.33333333, -0.66666667,  0.        ,  0.66666667,
        1.33333333, -2.        , -1.33333333, -0.66666667,  0.        ,
        0.66666667,  1.33333333, -2.        , -1.33333333, -0.66666667,
        0.        ,  0.66666667,  1.33333333, -2.        , -1.33333333,
       -0.66666667,  0.        ,  0.66666667,  1.33333333, -2.        ,
       -1.33333333, -0.66666667,  0.        ,  0.66666667,  1.33333333]), array([ 1.53696831e+01+0.00000000e+00j,  1.89871933e+04+0.00000000e+00j,
        1.88953405e+05+0.00000000e+00j,  3.69392758e+05+0.00000000e+00j,
        1.88953405e+05+0.00000000e+00j,  1.89871933e+04+0.00000000e+00j,
        1.89871933e+04+0.00000000e+00j,  2.34561447e+07+0.00000000e+00j,
        2.33426728e+08+0.00000000e+00j,  4.56335480e+08+0.00000000e+00j,
        2.33426728e+08+0.00000000e+00j,  2.34561447e+07+0.00000000e+00j,
        1.88953405e+05+0.00000000e+00j,  2.33426728e+08+0.00000000e+00j,
        2.32297499e+09+0.00000000e+00j,  4.54127903e+09+0.00000000e+00j,
        2.32297499e+09+0.00000000e+00j,  2.33426728e+08+0.00000000e+00j,
        3.69392758e+05+0.00000000e+00j,  4.56335480e+08+0.00000000e+00j,
        4.54127903e+09+0.00000000e+00j,  8.87793254e+09+0.00000000e+00j,
        4.54127903e+09+0.00000000e+00j,  4.56335480e+08+0.00000000e+00j,
        1.88953405e+05+0.00000000e+00j,  2.33426728e+08+0.00000000e+00j,
        2.32297499e+09+0.00000000e+00j,  4.54127903e+09+0.00000000e+00j,
        2.32297499e+09+0.00000000e+00j,  2.33426728e+08+0.00000000e+00j,
        1.89871933e+04+0.00000000e+00j,  2.34561447e+07+0.00000000e+00j,
        2.33426728e+08+0.00000000e+00j,  4.56335480e+08+0.00000000e+00j,
        2.33426728e+08+0.00000000e+00j,  2.34561447e+07+0.00000000e+00j,
        6.50565534e+04+6.96317590e+04j,  1.44083431e+06+1.54216328e+06j,
        4.55440322e+06+4.87469887e+06j,  3.66461498e+06+3.92233486e+06j,
        6.88458811e+05+7.36875773e+05j,  9.80283881e+03+1.04922390e+04j,
        3.72168203e+06+3.98341524e+06j,  8.24256267e+07+8.82223400e+07j,
        2.60543170e+08+2.78866282e+08j,  2.09641166e+08+2.24384515e+08j,
        3.93845763e+07+4.21543593e+07j,  5.60789761e+05+6.00228194e+05j,
        1.83042628e+07+1.95915392e+07j,  4.05392056e+08+4.33901898e+08j,
        1.28142345e+09+1.37154160e+09j,  1.03107331e+09+1.10358518e+09j,
        1.93704252e+08+2.07326812e+08j,  2.75811934e+06+2.95208847e+06j,
        2.13715158e+07+2.28745016e+07j,  4.73323773e+08+5.06611020e+08j,
        1.49615212e+09+1.60137140e+09j,  1.20385069e+09+1.28851341e+09j,
        2.26163354e+08+2.42068652e+08j,  3.22029854e+06+3.44677116e+06j,
        6.29828664e+06+6.74122367e+06j,  1.39490751e+08+1.49300660e+08j,
        4.40923096e+08+4.71931714e+08j,  3.54780485e+08+3.79730987e+08j,
        6.66514088e+07+7.13387752e+07j,  9.49037191e+05+1.01577974e+06j,
        2.48497032e+05+2.65972981e+05j,  5.50356623e+06+5.89061328e+06j,
        1.73964900e+07+1.86199258e+07j,  1.39977588e+07+1.49821735e+07j,
        2.62971157e+06+2.81465022e+06j,  3.74439809e+04+4.00772885e+04j,
        1.63487960e+01+0.00000000e+00j,  2.01967568e+04+0.00000000e+00j,
        2.00990526e+05+0.00000000e+00j,  3.92924618e+05+0.00000000e+00j,
        2.00990526e+05+0.00000000e+00j,  2.01967568e+04+0.00000000e+00j,
        2.01967568e+04+0.00000000e+00j,  2.49503990e+07+0.00000000e+00j,
        2.48296985e+08+0.00000000e+00j,  4.85405954e+08+0.00000000e+00j,
        2.48296985e+08+0.00000000e+00j,  2.49503990e+07+0.00000000e+00j,
        2.00990526e+05+0.00000000e+00j,  2.48296985e+08+0.00000000e+00j,
        2.47095819e+09+0.00000000e+00j,  4.83057746e+09+0.00000000e+00j,
        2.47095819e+09+0.00000000e+00j,  2.48296985e+08+0.00000000e+00j,
        3.92924618e+05+0.00000000e+00j,  4.85405954e+08+0.00000000e+00j,
        4.83057746e+09+0.00000000e+00j,  9.44349389e+09+0.00000000e+00j,
        4.83057746e+09+0.00000000e+00j,  4.85405954e+08+0.00000000e+00j,
        2.00990526e+05+0.00000000e+00j,  2.48296985e+08+0.00000000e+00j,
        2.47095819e+09+0.00000000e+00j,  4.83057746e+09+0.00000000e+00j,
        2.47095819e+09+0.00000000e+00j,  2.48296985e+08+0.00000000e+00j,
        2.01967568e+04+0.00000000e+00j,  2.49503990e+07+0.00000000e+00j,
        2.48296985e+08+0.00000000e+00j,  4.85405954e+08+0.00000000e+00j,
        2.48296985e+08+0.00000000e+00j,  2.49503990e+07+0.00000000e+00j,
       -2.89784935e+01-2.25912281e+02j, -2.23908706e+04-1.74556094e+05j,
       -2.11475289e+05-1.64863176e+06j, -3.99143830e+05-3.11166945e+06j,
       -1.97089760e+05-1.53648419e+06j, -1.87779394e+04-1.46390188e+05j,
       -3.22170681e+02-2.51159755e+03j, -2.48932265e+05-1.94064110e+06j,
       -2.35109316e+06-1.83287933e+07j, -4.43751292e+06-3.45942298e+07j,
       -2.19116091e+06-1.70819838e+07j, -2.08765218e+05-1.62750443e+06j,
       -6.77462309e+02-5.28140137e+03j, -5.23456158e+05-4.08079096e+06j,
       -4.94389184e+06-3.85418890e+07j, -9.33122698e+06-7.27449399e+07j,
       -4.60758541e+06-3.59200911e+07j, -4.38992668e+05-3.42232541e+06j,
       -3.72502233e+02-2.90397529e+03j, -2.87822046e+05-2.24382040e+06j,
       -2.71839588e+06-2.11922339e+07j, -5.13076941e+06-3.99987604e+07j,
       -2.53347800e+06-1.97506400e+07j, -2.41379848e+05-1.88176352e+06j,
       -4.16466603e+01-3.24671536e+02j, -3.21792084e+04-2.50864606e+05j,
       -3.03923305e+05-2.36934356e+06j, -5.73632563e+05-4.47195919e+06j,
       -2.83249034e+05-2.20816983e+06j, -2.69868571e+04-2.10385762e+05j,
       -7.89426854e-02-6.15426129e-01j, -6.09968027e+01-4.75522539e+02j,
       -5.76097139e+02-4.49117269e+03j, -1.08734037e+03-8.47675336e+03j,
       -5.36908342e+02-4.18566231e+03j, -5.11545212e+01-3.98793490e+02j,
       -2.39491817e+03-9.14328375e+02j, -1.50142102e+05-5.73210333e+04j,
       -7.65490990e+05-2.92248037e+05j, -9.19663696e+05-3.51107868e+05j,
       -2.80007257e+05-1.06900763e+05j, -1.17618863e+04-4.49043583e+03j,
       -7.12338996e+04-2.71955746e+04j, -4.46579242e+06-1.70494373e+06j,
       -2.27685893e+07-8.69255891e+06j, -2.73542671e+07-1.04432723e+07j,
       -8.32847196e+06-3.17963191e+06j, -3.49842861e+05-1.33562499e+05j,
       -2.61985248e+05-1.00020347e+05j, -1.64243674e+07-6.27047107e+06j,
       -8.37387053e+07-3.19696411e+07j, -1.00603989e+08-3.84084446e+07j,
       -3.06305959e+07-1.16941043e+07j, -1.28665803e+06-4.91218430e+05j,
       -2.40566040e+05-9.18429526e+04j, -1.50815553e+07-5.75781424e+06j,
       -7.68924542e+07-2.93558893e+07j, -9.23788777e+07-3.52682735e+07j,
       -2.81263208e+07-1.07380258e+07j, -1.18146433e+06-4.51057734e+05j,
       -5.33261451e+04-2.03587781e+04j, -3.34312028e+06-1.27633160e+06j,
       -1.70447091e+07-6.50730425e+06j, -2.04775763e+07-7.81789930e+06j,
       -6.23474647e+06-2.38029244e+06j, -2.61894565e+05-9.99857261e+04j,
       -1.14894686e+03-4.38643261e+02j, -7.20297248e+04-2.74994035e+04j,
       -3.67239466e+05-1.40204149e+05j, -4.41202847e+05-1.68441781e+05j,
       -1.34331712e+05-5.12849657e+04j, -5.64269062e+03-2.15425822e+03j,
        1.62055063e+01+0.00000000e+00j,  2.00197415e+04+0.00000000e+00j,
        1.99228936e+05+0.00000000e+00j,  3.89480814e+05+0.00000000e+00j,
        1.99228936e+05+0.00000000e+00j,  2.00197415e+04+0.00000000e+00j,
        2.00197415e+04+0.00000000e+00j,  2.47317203e+07+0.00000000e+00j,
        2.46120776e+08+0.00000000e+00j,  4.81151595e+08+0.00000000e+00j,
        2.46120776e+08+0.00000000e+00j,  2.47317203e+07+0.00000000e+00j,
        1.99228936e+05+0.00000000e+00j,  2.46120776e+08+0.00000000e+00j,
        2.44930138e+09+0.00000000e+00j,  4.78823967e+09+0.00000000e+00j,
        2.44930138e+09+0.00000000e+00j,  2.46120776e+08+0.00000000e+00j,
        3.89480814e+05+0.00000000e+00j,  4.81151595e+08+0.00000000e+00j,
        4.78823967e+09+0.00000000e+00j,  9.36072602e+09+0.00000000e+00j,
        4.78823967e+09+0.00000000e+00j,  4.81151595e+08+0.00000000e+00j,
        1.99228936e+05+0.00000000e+00j,  2.46120776e+08+0.00000000e+00j,
        2.44930138e+09+0.00000000e+00j,  4.78823967e+09+0.00000000e+00j,
        2.44930138e+09+0.00000000e+00j,  2.46120776e+08+0.00000000e+00j,
        2.00197415e+04+0.00000000e+00j,  2.47317203e+07+0.00000000e+00j,
        2.46120776e+08+0.00000000e+00j,  4.81151595e+08+0.00000000e+00j,
        2.46120776e+08+0.00000000e+00j,  2.47317203e+07+0.00000000e+00j,
        5.11842921e+02+9.27008970e+01j,  1.76101589e+04+3.18941117e+03j,
        6.94610720e+04+1.25802339e+04j,  6.76854197e+04+1.22586420e+04j,
        1.61283925e+04+2.92104549e+03j,  4.09693014e+02+7.42003226e+01j,
        8.27206418e+04+1.49817012e+04j,  2.84603653e+06+5.15451379e+05j,
        1.12258356e+07+2.03313357e+06j,  1.09388665e+07+1.98116002e+06j,
        2.60656332e+06+4.72079905e+05j,  6.62118546e+04+1.19917617e+04j,
        5.75358778e+05+1.04204381e+05j,  1.97954472e+07+3.58519311e+06j,
        7.80806687e+07+1.41413464e+07j,  7.60846713e+07+1.37798473e+07j,
        1.81298047e+07+3.28352527e+06j,  4.60532836e+05+8.34080247e+04j,
        8.75507391e+05+1.58564898e+05j,  3.01221795e+07+5.45548828e+06j,
        1.18813174e+08+2.15184920e+07j,  1.15775920e+08+2.09684089e+07j,
        2.75876178e+07+4.99644874e+06j,  7.00779960e+05+1.26919663e+05j,
        3.45751878e+05+6.26198155e+04j,  1.18957307e+07+2.15445962e+06j,
        4.69212236e+07+8.49799682e+06j,  4.57217636e+07+8.28076021e+06j,
        1.08947917e+07+1.97317756e+06j,  2.76749220e+05+5.01226058e+04j,
        2.30837727e+04+4.18074833e+03j,  7.94206366e+05+1.43840306e+05j,
        3.13264780e+06+5.67360119e+05j,  3.05256707e+06+5.52856538e+05j,
        7.27379694e+05+1.31737194e+05j,  1.84768803e+04+3.34638482e+03j,
        7.70985688e+04-7.69146690e+02j,  1.05214036e+06-1.04963074e+04j,
        2.51801412e+06-2.51200801e+04j,  1.56888750e+06-1.56514530e+04j,
        2.09577575e+05-2.09077679e+03j,  9.74258672e+02-9.71934816e+00j,
        1.19598326e+06-1.19313053e+04j,  1.63212142e+07-1.62822839e+05j,
        3.90604233e+07-3.89672542e+05j,  2.43371986e+07-2.42791481e+05j,
        3.25104960e+06-3.24329501e+04j,  1.51130829e+04-1.50770343e+02j,
        3.09234189e+06-3.08496586e+04j,  4.22002348e+07-4.20995764e+05j,
        1.00994878e+08-1.00753979e+06j,  6.29264148e+07-6.27763192e+05j,
        8.40593443e+06-8.38588413e+04j,  3.90764828e+04-3.89832754e+02j,
        2.07227487e+06-2.06733196e+04j,  2.82796952e+07-2.82122409e+05j,
        6.76798216e+07-6.75183879e+05j,  4.21689557e+07-4.20683719e+05j,
        5.63307917e+06-5.61964284e+04j,  2.61863715e+04-2.61239103e+02j,
        3.05965972e+05-3.05236165e+03j,  4.17542314e+06-4.16546369e+04j,
        9.99274890e+06-9.96891363e+04j,  6.22613618e+06-6.21128525e+04j,
        8.31709428e+05-8.29725588e+03j,  3.86634936e+03-3.85712712e+01j,
        2.08342636e+03-2.07845685e+01j,  2.84318761e+04-2.83640588e+02j,
        6.80440256e+04-6.78817232e+02j,  4.23958786e+04-4.22947536e+02j,
        5.66339234e+03-5.64988370e+01j,  2.63272876e+01-2.62644903e-01j,
       -8.48177867e+01-1.09394264e+02j, -3.18315253e+03-4.10549061e+03j,
       -1.30718136e+04-1.68594522e+04j, -1.31748913e+04-1.69923973e+04j,
       -3.26915509e+03-4.21641294e+03j, -9.07415545e+01-1.17034480e+02j,
       -9.88073837e+03-1.27437433e+04j, -3.70817238e+05-4.78263820e+05j,
       -1.52278402e+06-1.96402008e+06j, -1.53479192e+06-1.97950735e+06j,
       -3.80835994e+05-4.91185573e+05j, -1.05708200e+04-1.36337804e+04j,
       -6.24821671e+04-8.05867605e+04j, -2.34491227e+06-3.02436506e+06j,
       -9.62952786e+06-1.24197430e+07j, -9.70546146e+06-1.25176788e+07j,
       -2.40826721e+06-3.10607749e+06j, -6.68459906e+04-8.62150288e+04j,
       -8.86027902e+04-1.14275995e+05j, -3.32520108e+06-4.28869861e+06j,
       -1.36551448e+07-1.76118072e+07j, -1.37628223e+07-1.77506850e+07j,
       -3.41504151e+06-4.40457086e+06j, -9.47909069e+04-1.22257157e+05j,
       -3.24327620e+04-4.18303548e+04j, -1.21717899e+06-1.56986412e+06j,
       -4.99842114e+06-6.44674450e+06j, -5.03783617e+06-6.49758028e+06j,
       -1.25006479e+06-1.61227878e+06j, -3.46979019e+04-4.47518330e+04j,
       -1.90309898e+03-2.45453365e+03j, -7.14219808e+04-9.21169404e+04j,
       -2.93298800e+05-3.78283937e+05j, -2.95611606e+05-3.81266893e+05j,
       -7.33516631e+04-9.46057600e+04j, -2.03601351e+03-2.62596099e+03j,
        1.98329554e+01+0.00000000e+00j,  2.45009709e+04+0.00000000e+00j,
        2.43824446e+05+0.00000000e+00j,  4.76662404e+05+0.00000000e+00j,
        2.43824446e+05+0.00000000e+00j,  2.45009709e+04+0.00000000e+00j,
        2.45009709e+04+0.00000000e+00j,  3.02676815e+07+0.00000000e+00j,
        3.01212580e+08+0.00000000e+00j,  5.88852820e+08+0.00000000e+00j,
        3.01212580e+08+0.00000000e+00j,  3.02676815e+07+0.00000000e+00j,
        2.43824446e+05+0.00000000e+00j,  3.01212580e+08+0.00000000e+00j,
        2.99755429e+09+0.00000000e+00j,  5.86004175e+09+0.00000000e+00j,
        2.99755429e+09+0.00000000e+00j,  3.01212580e+08+0.00000000e+00j,
        4.76662404e+05+0.00000000e+00j,  5.88852820e+08+0.00000000e+00j,
        5.86004175e+09+0.00000000e+00j,  1.14560358e+10+0.00000000e+00j,
        5.86004175e+09+0.00000000e+00j,  5.88852820e+08+0.00000000e+00j,
        2.43824446e+05+0.00000000e+00j,  3.01212580e+08+0.00000000e+00j,
        2.99755429e+09+0.00000000e+00j,  5.86004175e+09+0.00000000e+00j,
        2.99755429e+09+0.00000000e+00j,  3.01212580e+08+0.00000000e+00j,
        2.45009709e+04+0.00000000e+00j,  3.02676815e+07+0.00000000e+00j,
        3.01212580e+08+0.00000000e+00j,  5.88852820e+08+0.00000000e+00j,
        3.01212580e+08+0.00000000e+00j,  3.02676815e+07+0.00000000e+00j]))

In [209]:
# vg = vg[:10]
# ug = ug[:10]
# visg = visg[:10]

In [216]:
gv = np.zeros((Nu, Nv), dtype=complex) #output array


N = ug.shape[0]
temp = 0

undxs = []
vndxs = []


for i in range(N):
    # compute the location for the visibility in the visibility cube
    temp = (ug[i] - umin)/du + 0.5
    undx = int(temp)
    temp = (vg[i] - vmin)/dv + 0.5
    vndx = int(temp)
    

    if (undx>=0 and undx<Nu) and (vndx>=0 and vndx<Nv):
        gv[undx, vndx] = gv[undx, vndx] + visg[i]
        
    
#     now compute the location for the -u,-v,-l2 visibility, which is
#     equal to the complex conj of the u,v,l2 visibility if we
#     assume that the individual Stokes images in Faraday space are real

    
    if hflag_u or hflag_v:
        if hflag_u:
            temp = (-1.*ug[i] - umin)/du + 0.5
            undx = int(temp)
            undxs.append(undx)

        if hflag_v:
            temp = (-1.*vg[i] - vmin)/dv + 0.5
            vndx = int(temp)
            vndxs.append(vndx)

        if (undx>=0 and undx<Nu) and (vndx>=0 and vndx<Nv):
#             print(gv[undx, vndx], visg[i].conjugate())
            gv[undx, vndx] = gv[undx, vndx] + visg[i].conjugate()

2.06811891e+08+0.00000000e+00j

(368638072232.6662+74861774.93049999j)


In [220]:
a = [2.06811891e+08+0.00000000e+00j 2.07641840e+09+1.95915392e+07j
 3.33953759e+09+1.37154160e+09j 2.07211190e+09-1.49821735e+07j
 2.19779783e+10+1.60110542e+09j 3.26196483e+09+1.28851341e+09j
 4.37827218e+09+3.79730987e+08j 2.26996352e+08-2.42068652e+08j
 2.19779783e+10-1.60110542e+09j 1.93787957e+08-2.07326812e+08j
 3.33953759e+09-1.37154160e+09j 2.07641840e+09-1.95915392e+07j]

SyntaxError: invalid syntax (<ipython-input-220-0caf93ba7713>, line 1)

In [221]:
a, b = (array([2.06811891e+08+0.00000000e+00j, 2.07641840e+09+1.95915392e+07j,
       3.33953759e+09+1.37154160e+09j, 2.07211190e+09-1.49821735e+07j,
       2.19779783e+10+1.60110542e+09j, 3.26196483e+09+1.28851341e+09j,
       4.37827218e+09+3.79730987e+08j, 2.26996352e+08-2.42068652e+08j,
       2.19779783e+10-1.60110542e+09j, 1.93787957e+08-2.07326812e+08j,
       3.33953759e+09-1.37154160e+09j, 2.07641840e+09-1.95915392e+07j]), array([2.06811891e+08+0.00000000e+00j, 2.07641840e+09+1.95915392e+07j,
       3.33953759e+09+1.37154160e+09j, 2.07211190e+09-1.49821735e+07j,
       2.19779783e+10+1.60110542e+09j, 3.26196483e+09+1.28851341e+09j,
       4.37827218e+09+3.79730987e+08j, 2.26996352e+08-2.42068652e+08j,
       2.19779783e+10-1.60110542e+09j, 1.93787957e+08-2.07326812e+08j,
       3.33953759e+09-1.37154160e+09j, 2.07641840e+09-1.95915392e+07j]))

In [222]:

a == b

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [217]:

gvi = np.zeros((Nu, Nv), dtype=complex) #output array

undxi = ((ug - umin) / du + 0.5).astype(int)
vndxi = ((vg - vmin) / du + 0.5).astype(int)

mask = ((undxi >= 0) & (undxi < Nu)) & ((vndxi >= 0) & (vndxi < Nv))

np.add.at(gvi, (undxi[mask], vndxi[mask]), visg[mask])# print(gv.sum())
if hflag_u or hflag_v:
    if hflag_u:
        undxi = ((-1.*ug - umin)/du + 0.5).astype(int)
    if hflag_v:
        vndxi = ((-1.*vg - vmin)/dv + 0.5).astype(int)
        
    
        
    mask = ((undxi >= 0) & (undxi < Nu)) & ((vndxi >= 0) & (vndxi < Nv))    
        
    np.add.at(gvi, (undxi[mask], vndxi[mask]), visg.conjugate()[mask])
    
        
print(gvi.sum())        
    


(368638072232.6662+74861774.93049999j)


In [117]:
(undxi == undxs).all()

True

In [119]:
(vndxi == vndxs).all()

True

In [106]:
(gvi == gv).all()

True

In [24]:
np.add.at(gv, (undxi, vndxi), visg)

In [25]:
gv.sum()

-3.683982668976944j

In [26]:
gv.sum()

-3.683982668976944j

In [27]:
visg.sum()

-3.683982668976943j

In [28]:
a = np.zeros(6, dtype=complex)

In [29]:
visg = np.random.randn(6) + np.random.randn(6) * 1j

In [30]:
visg.sum()

(5.425327791625499-3.272533379977702j)

In [31]:
(a+ visg).sum()

(5.425327791625499-3.272533379977702j)

In [234]:
from gridding_python_improved.gridding import get_beta, inv_gcf_kaiser

KeyboardInterrupt: 

In [294]:
def inv_gcf_kaiseri(x, dk, W, beta):
    
    temp1 = np.pi*np.pi*W*W*dk*dk*x*x
    temp2 = beta*beta

    temp = np.sqrt(temp2 - temp1)
    c0 = (np.exp(beta)-np.exp(-1.*beta))/2./beta
    c = (np.exp(temp) - np.exp(-1.*temp))/2./temp
    
    tdg = temp1 > temp2
    
    if tdg.any():
        temp = temp1[tdg] - temp2
        c[tdg] = -0.5*(np.exp(-1.*temp) - np.exp(temp))/temp

#         c[tdg] = -0.5*(np.exp(-1.*np.sqrt(temp))) - np.exp(np.sqrt(temp)))/np.sqrt(temp1 - temp2)

    return c/c0


def get_grid_corr_2di(dx, Nx, xmin, dy, Ny, ymin, du, dv, W, alpha):

    x = np.arange(Nx)*dx + xmin
    y = np.arange(Ny)*dy + ymin

    # see Beatty et al. (2005)
    beta = get_beta(W, alpha)
    
    x_inv_kaiser = inv_gcf_kaiseri(x, du, W, beta)
    y_inv_kaiser = inv_gcf_kaiseri(y, dv, W, beta)
    
    gridcorr = np.outer(x_inv_kaiser, y_inv_kaiser)
    return gridcorr


#     for i in range(Nx):
#         for j in range(Ny):
#             gridcorr[i,j] = inv_gcf_kaiser(x[i], du, W, beta)*\
#                 inv_gcf_kaiser(y[j], dv, W, beta)

#     return gridcorr


In [295]:
dx, dy = (0.0004347826086956522, 0.0004347826086956522)
Nx, Ny = (2300, 2300)
xmin, ymin = (-0.5, -0.5)
gci = get_grid_corr_2di(dx, Nx, xmin, dy, Ny, ymin, du, dv, W, alpha)

In [292]:
dx, dy = (0.0004347826086956522, 0.0004347826086956522)
Nx, Ny = (2300, 2300)
xmin, ymin = (-0.5, -0.5)
gc = get_grid_corr_2d(dx, Nx, xmin, dy, Ny, ymin, du, dv, W, alpha)

In [296]:
np.isclose(gci, gc).all()

True